In [60]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import csv
from implementations import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
File_test = open('test.csv')
File_train = open('train.csv')

data_test = np.array(list(csv.reader(File_test)))
data_train = np.array(list(csv.reader(File_train)))

File_test.close()
File_train.close()

In [62]:
X = np.array(data_train[1:,2:]).astype(np.float)
X = np.where(X == -999., np.nan, X)
X, means, stds = standardize(X)

In [63]:
features = data_train[0,2:]
labels = np.array(data_train[1:,1])
labels = np.where(labels == 'b', 1, 0)

In [64]:
X.shape

(250000, 30)

In [66]:
X = np.hstack((np.ones(X.shape[0]), X))

ValueError: all the input arrays must have same number of dimensions

In [67]:
X

array([[ 0.28991353,  0.06833197,  0.40768027, ...,  0.61614788,
        -1.36131161,  0.4125105 ],
       [ 0.68202131,  0.55250482,  0.54013641, ...,         nan,
                nan, -0.27381996],
       [        nan,  3.19515553,  1.09655998, ...,         nan,
                nan, -0.29396985],
       ...,
       [-0.28624947,  0.31931645, -0.13086367, ...,         nan,
                nan, -0.31701723],
       [-0.46960659, -0.84532397, -0.30297338, ...,         nan,
                nan, -0.74543941],
       [        nan,  0.66533608, -0.25352276, ...,         nan,
                nan, -0.74543941]])